In [ ]:
import numpy as np
import scrublet as scr
import scanpy as sc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os
from collections import Counter
import anndata as ann
import bbknn
import cellrank as cr
import scanpy as sc
import scvelo as scv
import loompy

In [ ]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

In [ ]:
loom_list = pd.read_csv('./loom/loom_list.csv')

In [ ]:
loom_list = loom_list.iloc[:, 0].tolist()

In [ ]:
loompy.combine(loom_list, output_file = 'DNT_combined.loom')

In [ ]:
adata = scv.read_loom("./DNT_combined.loom")

In [ ]:
adata

In [ ]:
adata.obs['barcodes'] = adata.obs.index.str.split(':').str[1]

In [ ]:
adata.obs['barcodes'] = adata.obs['barcodes'].str.replace('x$', '', regex=True)

In [ ]:
adata.obs['sample'] = adata.obs.index.str.split('_possorted').str[0]

In [ ]:
adata.obs['cellID'] = adata.obs['sample'] + '_' + adata.obs.barcodes

In [ ]:
adata_DNT = sc.read_h5ad("./RDS/pancancer_DNT_symbol_regress_leiden_drop_v7_.h5ad")

In [ ]:
adata_DNT.obs

In [ ]:
adata.obs = pd.merge(adata.obs, adata_DNT.obs, on='cellID', how='left', suffixes=('', '_DNT'))

In [ ]:
adata.obs = pd.merge(adata.obs, adata_DNT.obs, on='cellID', how='left', suffixes=('', '_DNT'))

In [ ]:
adata.obs.index = adata.obs.cellID

In [ ]:
adata_sample = adata_DNT[adata_DNT.obs.cellID.isin(adata.obs.index)]

In [ ]:
adata_sample.obs.index = adata_sample.obs.cellID

In [ ]:
adata_sample = adata_sample[adata.obs.index,:]

In [ ]:
adata.obsm['X_umap'] = adata_sample.obsm['X_umap']

In [ ]:
adata.uns['DNT_C_colors'] = adata_sample.uns['DNT_C_colors']

In [ ]:
adata.var_names_make_unique()

In [ ]:
common_genes = adata.var_names.intersection(adata_sample.var_names)

In [ ]:
adata = adata[:,adata.var_names.isin(common_genes)]

In [ ]:
adata.write_h5ad("./h5ad/DNT_velocity_input.h5ad")

In [ ]:
scv.pp.filter_and_normalize(adata, min_shared_counts=30, min_shared_cells=30, log=True)

In [ ]:
sc.tl.pca(adata)
sc.pp.neighbors(adata, n_pcs=30, n_neighbors=30, random_state=0)
scv.pp.moments(adata, n_pcs=None, n_neighbors=None)

In [ ]:
scv.tl.recover_dynamics(adata,n_jobs=80)

In [ ]:
scv.tl.velocity(adata, mode="dynamical",min_r2=None)

In [ ]:
scv.tl.velocity_graph(adata)

In [ ]:
scv.settings.set_figure_params('scvelo', dpi=300)
scv.pl.velocity_embedding_stream(adata, basis='umap',color='DNT_C',smooth=0.8, min_mass=2.5,
                                 legend_loc='right margin',legend_fontsize=5,
                                 save="DNT_stream.pdf", figsize=(5,5))

In [ ]:
abT = adata[adata.obs.DNT_C.isin(['T1_RGS1+CREM+','T2_LEF1+CCR7+','T3_CD40LG+CCR6+','T4_CXCL13+PDCD1+','T5_FOXP3+IL2RA+','T6_CXCL1+CXCL8+'])]

In [ ]:
gdT = adata[~adata.obs.DNT_C.isin(['T1_RGS1+CREM+','T2_LEF1+CCR7+','T3_CD40LG+CCR6+','T4_CXCL13+PDCD1+','T5_FOXP3+IL2RA+','T6_CXCL1+CXCL8+'])]

In [ ]:
abT.var_names_make_unique()
gdT.var_names_make_unique()

In [ ]:
scv.pp.filter_and_normalize(abT, min_shared_counts=30, min_shared_cells=30, log=True)
scv.pp.filter_and_normalize(gdT, min_shared_counts=30, min_shared_cells=30, log=True)

In [ ]:
sc.tl.pca(abT)
sc.pp.neighbors(abT, n_pcs=30, n_neighbors=30, random_state=0)
scv.pp.moments(abT, n_pcs=None, n_neighbors=None)

In [ ]:
sc.tl.pca(gdT)
sc.pp.neighbors(gdT, n_pcs=30, n_neighbors=30, random_state=0)
scv.pp.moments(gdT, n_pcs=None, n_neighbors=None)

In [ ]:
scv.tl.recover_dynamics(abT, n_jobs=80)
scv.tl.recover_dynamics(gdT, n_jobs=80)
scv.tl.velocity(abT, mode="dynamical")
scv.tl.velocity(gdT, mode="dynamical")
scv.tl.velocity_graph(abT)
scv.tl.velocity_graph(gdT)

In [ ]:
scv.settings.set_figure_params('scvelo', dpi=300)
scv.pl.velocity_embedding_stream(abT, basis='umap',color='DNT_C',smooth=0.8, min_mass=2.5,
                                 legend_loc='right margin',legend_fontsize=5,
                                 save="abT_stream.png", figsize=(5,5))

In [ ]:
scv.settings.set_figure_params('scvelo', dpi=300)
scv.pl.velocity_embedding_stream(gdT, basis='umap',color='DNT_C',smooth=0.8, min_mass=2.5,
                                 legend_loc='right margin',legend_fontsize=5,
                                 save="gdT_stream.png", figsize=(5,5))